In [8]:
import pandas as pd
import openai
import os
import tiktoken

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# ChatGPT API tarafini da izleyip bitir. Ondan sonra burada Completion API ile yapilanlari ChatGPT API ile dene. Mesela, December 2022 degerlemelerini 
# kendi cevabina eklemesini isteyebilirsin.